<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Crawling-Coupang-Item-Package" data-toc-modified-id="Crawling-Coupang-Item-Package-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Crawling Coupang Item Package</a></span><ul class="toc-item"><li><span><a href="#1.-coupang-모듈-만들기" data-toc-modified-id="1.-coupang-모듈-만들기-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>1. coupang 모듈 만들기</a></span></li><li><span><a href="#2.-setup.py-파일-생성" data-toc-modified-id="2.-setup.py-파일-생성-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>2. setup.py 파일 생성</a></span></li><li><span><a href="#3.-패키지-설치" data-toc-modified-id="3.-패키지-설치-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>3. 패키지 설치</a></span></li><li><span><a href="#4.-패키지-설치-확인" data-toc-modified-id="4.-패키지-설치-확인-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>4. 패키지 설치 확인</a></span></li><li><span><a href="#5.-패키지-사용" data-toc-modified-id="5.-패키지-사용-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>5. 패키지 사용</a></span><ul class="toc-item"><li><span><a href="#1.-객체-생성" data-toc-modified-id="1.-객체-생성-1.5.1"><span class="toc-item-num">1.5.1&nbsp;&nbsp;</span>1. 객체 생성</a></span></li><li><span><a href="#2.-페이지-이동" data-toc-modified-id="2.-페이지-이동-1.5.2"><span class="toc-item-num">1.5.2&nbsp;&nbsp;</span>2. 페이지 이동</a></span></li><li><span><a href="#3.-현재-페이지-크롤링" data-toc-modified-id="3.-현재-페이지-크롤링-1.5.3"><span class="toc-item-num">1.5.3&nbsp;&nbsp;</span>3. 현재 페이지 크롤링</a></span></li><li><span><a href="#4.-키워드-변경" data-toc-modified-id="4.-키워드-변경-1.5.4"><span class="toc-item-num">1.5.4&nbsp;&nbsp;</span>4. 키워드 변경</a></span></li><li><span><a href="#5.-시작-페이지와-끝-페이지를-입력하면-그-페이지에-해당되는-데이터를-크롤링" data-toc-modified-id="5.-시작-페이지와-끝-페이지를-입력하면-그-페이지에-해당되는-데이터를-크롤링-1.5.5"><span class="toc-item-num">1.5.5&nbsp;&nbsp;</span>5. 시작 페이지와 끝 페이지를 입력하면 그 페이지에 해당되는 데이터를 크롤링</a></span></li><li><span><a href="#6.-객체-안에-있는-썸네일-링크의-이미지-저장" data-toc-modified-id="6.-객체-안에-있는-썸네일-링크의-이미지-저장-1.5.6"><span class="toc-item-num">1.5.6&nbsp;&nbsp;</span>6. 객체 안에 있는 썸네일 링크의 이미지 저장</a></span></li><li><span><a href="#7.-webdriver-객체-닫기" data-toc-modified-id="7.-webdriver-객체-닫기-1.5.7"><span class="toc-item-num">1.5.7&nbsp;&nbsp;</span>7. webdriver 객체 닫기</a></span></li></ul></li></ul></li></ul></div>

## Crawling Coupang Item Package
- 시작 페이지와 종료 페이지와 키워드를 입력하여 아이템을 크롤링하는 모듈 만들기

- 크롤링하는 아이템 컬럼들
    - 제목
    - 가격
    - 평점 : 없으면 0
    - 평점수 : 없으면 0
    - 로켓배송여부 : true, false 데이터만 
    - 아이템 링크
    - 이미지 썸네일 링크

- 모듈에 있어야 하는 기능들
    - 아이템 정보 크롤링
    - 페이지 이동
    - 키워드 변경
    - 여러 페이지 크롤링 
    - 시작과 끝페이지, 키워드 입력하면 여러 페이지 크롤링
    - 크롤링한 썸네일 이미지 및 데이터 프레임 저장


### 1. coupang 모듈 만들기

In [1]:
%%writefile coupang.py

import requests
import os 
import pandas as pd
from selenium import webdriver
from fake_useragent import UserAgent

class Coupang:

    def __init__(self, keyword):

        # 크롬 웹 드라이버 객체 생성
        options = webdriver.ChromeOptions()
        options.add_argument("user-agent={}".format(UserAgent().chrome)) # user-agent 생성
        self.driver = webdriver.Chrome(options=options)

        # 생성자 함수에서 변수 선언
        self.__keyword = keyword

        # 키워드 입력 값이 있으면 페이지 이동
        url = "https://www.coupang.com/np/search?component=&q={}&channel=user".format(keyword)
        self.driver.get(url)

        # 결과 데이터 저장하는 변수 선언
        self.result_df = None

    def __download(self, path, title, link):
        
        title = title.replace("/", "") # path 지정에는 혹시라도 / 가 들어가면 안되니까
        
        # datas 디렉토리가 없으면 생성
        if not os.path.exists("./{}".format(path)):
            os.makedirs("./{}".format(path))
        
        # 다운로드
        headers = {'User-Agent': UserAgent().chrome}
        response = requests.get(link, headers=headers)
        with open("./{}/{}".format(path, title), "wb") as f:  # 검색명마다 디렉토리 생성해서 저장
            f.write(response.content)
    
    def save(self, path="datas"):
        titles = self.result_df["title"]
        links = self.result_df["thumnail_link"]
        for title, link in zip(titles, links):
            self.__download(path, title + ".png", link)
        self.result_df.to_csv("./{}/{}.csv".format(path, self.keyword), index=False)

    def crawling_one_page(self):
        elements = self.driver.find_elements_by_css_selector(".search-product")
        datas = []
        for element in elements:
            data = {
                "title": element.find_element_by_css_selector(".name").text,
                "price": element.find_element_by_css_selector(".price-value").text,
                "thumnail_link": element.find_element_by_css_selector(".search-product-wrap-img").get_attribute("src"),
                "is_rocket": element.get_attribute("data-is-rocket"),
                "link": element.find_element_by_css_selector("a.search-product-link").get_attribute("href"),
            }

            try:
                data["rating"] = element.find_element_by_css_selector(
                    ".other-info").text.split("\n")[0]
                data["rating_count"] = element.find_element_by_css_selector(
                    ".other-info").text.split("\n")[1][1:-1]
            except:
                data["rating"] = data["rating_count"] = ""

            datas.append(data)

        df = pd.DataFrame(datas)
        df["is_rocket"][df["is_rocket"] == ""] = "false"
        df["rating_count"][df["rating_count"] == ""] = "0"
        df["rating"][df["rating"] == ""] = "0"
        
        self.result_df = df
        self.result_df = self.result_df[["title", "price", "rating", "rating_count", "is_rocket", "link", "thumnail_link"]]
        
        return self.result_df
    
    def move_page(self, page):
        url = "https://www.coupang.com/np/search?q={}&channel=user&eventCategory=SRP&sorter=scoreDesc&listSize=36&isPriceRange=false&rating=0&page={}&rocketAll=false".format(
            self.keyword, page)
        self.driver.get(url)

    def close(self):
        self.driver.close()

    def run(self, start, end, keyword=None):
        if keyword:
            self.keyword_setter(keyword)
        dfs = []
        for page in range(start, end+1):
            print("keyword:{}, page:{} crawling ...".format(self.keyword, page))
            self.move_page(page)
            df = self.crawling_one_page()
            dfs.append(df)
        self.result_df = pd.concat(dfs, ignore_index=True, sort=False)
        return self.result_df

    def keyword_getter(self):
        return self.__keyword

    def keyword_setter(self, keyword):
        self.__keyword = keyword
        url = "https://www.coupang.com/np/search?q={}&channel=recent".format(
            self.__keyword)
        self.driver.get(url)

    keyword = property(keyword_getter, keyword_setter)
    
    def __repr__(self):
        return "<Coupang keyword={}>".format(self.keyword)

Overwriting coupang.py


### 2. setup.py 파일 생성

In [2]:
%%writefile setup.py

from setuptools import setup, find_packages

setup(
    name='dss.coupang',
    packages=find_packages(),
    include_package_data=True,
    version="0.0.1",
    author='Fast Campus, inc.',
    author_email="pdj1224@gmail.com",
    zip_safe=False,
)

Overwriting setup.py


### 3. 패키지 설치

In [3]:
!python setup.py develop

running develop
running egg_info
writing dss.coupang.egg-info/PKG-INFO
writing dependency_links to dss.coupang.egg-info/dependency_links.txt
writing top-level names to dss.coupang.egg-info/top_level.txt
reading manifest file 'dss.coupang.egg-info/SOURCES.txt'
writing manifest file 'dss.coupang.egg-info/SOURCES.txt'
running build_ext
Creating /usr/local/anaconda3/lib/python3.7/site-packages/dss.coupang.egg-link (link to .)
Adding dss.coupang 0.0.1 to easy-install.pth file

Installed /Users/rada/Documents/lecture/dss/dss_14/quiz/online_quiz
Processing dependencies for dss.coupang==0.0.1
Finished processing dependencies for dss.coupang==0.0.1


### 4. 패키지 설치 확인

In [4]:
!pip list | grep coupang

dss.coupang                       0.0.1               /Users/rada/Documents/lecture/dss/dss_14/quiz/online_quiz
dssf.coupang                      0.0.1               


### 5. 패키지 사용

In [5]:
import coupang as cp

#### 1. 객체 생성

In [6]:
coupang = cp.Coupang("bts")
coupang

<Coupang keyword=bts>

#### 2. 페이지 이동

In [7]:
coupang.move_page(17)

#### 3. 현재 페이지 크롤링
    - 결과를 데이터 프레임으로 리턴

In [8]:
coupang.crawling_one_page()
coupang.result_df.tail(2)

,title,price,rating,rating_count,is_rocket,link,thumnail_link
34,BTS 진 목베개 목쿠션,"19,100",0,0,false,https://www.coupang.com/vp/products/1980461176...,https://thumbnail6.coupangcdn.com/thumbnails/r...
35,LG Q7 BTS EDITION 케이스 Q727 LN_PPRS 플립 LG Q7 BT...,"33,330",0,0,false,https://www.coupang.com/vp/products/214817387?...,https://thumbnail7.coupangcdn.com/thumbnails/r...


#### 4. 키워드 변경
    - 객체의 변수를 변경하면 다른 키워드로 검색
    - 키워드가 변경된 후에는 1페이지로 이동 

In [9]:
coupang.keyword = "여름옷"

In [10]:
coupang.crawling_one_page()
coupang.result_df.tail(2)

,title,price,rating,rating_count,is_rocket,link,thumnail_link
38,캐럿 여성 주름 드레스,"21,900",4.5,9,true,https://www.coupang.com/vp/products/1658677679...,https://thumbnail10.coupangcdn.com/thumbnails/...
39,데일로엔 여성용 미키 브이넥 반팔 티셔츠,"15,900",4.5,79,false,https://www.coupang.com/vp/products/1353322288...,https://thumbnail7.coupangcdn.com/thumbnails/r...


#### 5. 시작 페이지와 끝 페이지를 입력하면 그 페이지에 해당되는 데이터를 크롤링
    - 키워드는 입력하면 해당 키워드로 크롤링하고, 입력되지 않으면 객체 안에 있는 키워드 변수로 크롤링

In [11]:
coupang.run(9, 11)
coupang.result_df.tail(2)

keyword:여름옷, page:9 crawling ...
keyword:여름옷, page:10 crawling ...
keyword:여름옷, page:11 crawling ...


,title,price,rating,rating_count,is_rocket,link,thumnail_link
106,빼빼 스모크밴딩 오프숄더 블라우스,"19,900",4.5,37,true,https://www.coupang.com/vp/products/1568844118...,https://thumbnail7.coupangcdn.com/thumbnails/r...
107,엔비룩 클래식 밴딩 원피스,"27,000",5.0,2,true,https://www.coupang.com/vp/products/1775635316...,https://thumbnail10.coupangcdn.com/thumbnails/...


In [12]:
coupang.run(19, 21, "선풍기")
coupang.result_df.tail(2)

keyword:선풍기, page:19 crawling ...
keyword:선풍기, page:20 crawling ...
keyword:선풍기, page:21 crawling ...


,title,price,rating,rating_count,is_rocket,link,thumnail_link
106,"제이닉스 51cm 3엽 대형 메탈 선풍기 JY-MF20, 단일 색상","47,800",4.5,2,false,https://www.coupang.com/vp/products/1487733699...,https://thumbnail8.coupangcdn.com/thumbnails/r...
107,[ALY_4855634] (주문선택 : 8인치-12V) 차량용 선풍기 차량 작은선풍...,"28,030",0,0,false,https://www.coupang.com/vp/products/1945294945...,https://thumbnail10.coupangcdn.com/thumbnails/...


#### 6. 객체 안에 있는 썸네일 링크의 이미지 저장

In [13]:
path = "datas"

In [14]:
coupang.save(path)

In [15]:
files = !ls datas
len(files), files[-2:]

(162,
 ['한솔일렉트로닉스 공업용 선풍기, HS-AJ205(20인치).png',
  '홍진테크사파이어 인더스트리얼 메탈 선풍기 HJ-12M, 브론즈(HJ-12MB).png'])

In [16]:
from PIL import Image as pli

In [17]:
"./{}/{}".format(path, files[0])

'./datas/BDO 비디오 휴대용 충전식 유모차선풍기, D1, 블루.png'

In [18]:
pli.open("./{}/{}".format(path, files[10]))

In [19]:
# 저장된 csv 파일 확인
df = pd.read_csv("./{}/{}.csv".format(path, coupang.keyword))
df.tail(2)

,title,price,rating,rating_count,is_rocket,link,thumnail_link
106,"제이닉스 51cm 3엽 대형 메탈 선풍기 JY-MF20, 단일 색상","47,800",4.5,2,False,https://www.coupang.com/vp/products/1487733699...,https://thumbnail8.coupangcdn.com/thumbnails/r...
107,[ALY_4855634] (주문선택 : 8인치-12V) 차량용 선풍기 차량 작은선풍...,"28,030",0.0,0,False,https://www.coupang.com/vp/products/1945294945...,https://thumbnail10.coupangcdn.com/thumbnails/...


#### 7. webdriver 객체 닫기

In [20]:
coupang.close()

In [21]:
# 패키지 삭제

In [22]:
!pip uninstall dss.coupang -y

Found existing installation: dss.coupang 0.0.1
Uninstalling dss.coupang-0.0.1:
  Successfully uninstalled dss.coupang-0.0.1


In [23]:
!pip list | grep coupang